In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'd:\\CampusX\\Miscellaneous topics\\Projects\\Text-Summarizer'

### Basic Configuration

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [4]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

### Configuration Updates

In [5]:
class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
import zipfile
from src.text_summarizer.logging import logger

### Components

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"File is downloaded")
        else:
            logger.info(f"File is already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [8]:
config = ConfigurationManager()
data_ingestion_config = config.get_data_ingestion_config()
data_ingestion = DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()

[2025-12-14 21:46:12,039: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-14 21:46:12,186: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-14 21:46:12,274: INFO: common: created directory at: artifacts]
[2025-12-14 21:46:12,276: INFO: common: created directory at: artifacts/data_ingestion]
[2025-12-14 21:46:18,115: INFO: 380177465: File is downloaded]
